<a href="https://colab.research.google.com/github/d-vinha/SPBD/blob/main/lab4/SPBD_Labs_spark1_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Spark Exercises


In [1]:
#@title Install Pyspark
!pip install --quiet pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


##1. Sorted Word Frequency

Create a [Spark](https://spark.apache.org/docs/latest/api/python/) program that counts the number of occurrences of each word in "Os Maias" novel, sorting them by frequency (the words with higher occurrence first).

Note that the sorting should be performed as a transformation (i.e. it should produce an RDD)...

In [2]:
#@title Download "Os Maias"
!wget -q -O os_maias.txt https://www.dropbox.com/s/n24v0z7y79np319/os_maias.txt?dl=0

1.1) Create a Spark program that counts the number of occurrences of each word in "Os Maias" novel, sorting them by frequency (the words with higher occurrence first).

Note that the sorting should be performed as a transformation (i.e. it should produce an RDD)...

In [3]:
import pyspark
import string

sc = pyspark.SparkContext('local[*]')
try:
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda word: word.lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \


  words = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a+b)


  sorted_words = words.sortBy(lambda x: x[1], ascending = False)

  for w,f in sorted_words.take(100):
      print("{}\t{}".format(w,f))

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

de	8459
o	7545
a	7208
e	6230
que	5280
um	3191
com	2872
do	2588
uma	2298
da	2208
não	2169
os	1888
para	1821
carlos	1798
em	1626
no	1560
se	1527
ao	1487
as	1455
na	1320
é	1296
ega	1125
como	1122
ele	1076
por	1068
era	970
mas	960
à	891
seu	885
mais	798
sua	770
ela	740
já	671
lhe	671
dos	623
muito	594
eu	586
depois	562
então	523
sobre	481
lá	479
das	475
tinha	441
maria	440
quando	437
ainda	433
sem	422
num	420
dâmaso	396
tudo	390
onde	385
foi	382
numa	380
estava	366
agora	365
bem	364
tão	357
seus	353
sr	351
disse	346
só	346
nos	339
entre	336
olhos	335
grande	332
também	331
casa	327
dum	324
afonso	320
logo	317
todo	314
vilaça	309
pela	294
ali	292
toda	291
sempre	289
pelo	286
me	281
tu	279
havia	278
fora	277
mão	269
maia	269
ou	268
assim	267
outro	267
até	262
dois	261
há	259
mesmo	258
homem	254
nem	252
isto	251
ar	248
craft	239
ás	237
está	237
duas	235
ser	228
meu	228


1.2) Create a Spark program that the top 10 most used words in "Os Maias" novel.

You should try to avoid sorting or finding the top-10 as actions. Your top-10 most used words should be a RDD at the end of the computation. Check *zipWithIndex* in [pyspark RDD](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html#pyspark.RDD) documentation.

In [ ]:
import pyspark
import string

sc = pyspark.SparkContext('local[*]')
try:
# Read text file, strip whitespace, convert to lowercase, and remove punctuation
  lines = sc.textFile('os_maias.txt') \
      .map( lambda line: line.strip() ) \
      .map( lambda word: word.lower() ) \
      .map( lambda line: line.translate(str.maketrans('', '', string.punctuation+'«»')) ) \

# Find top 10 words within each partition
  top10_words_partitions = lines.flatMap( lambda line: line.split() ) \
          .map( lambda word: (word, 1)) \
          .reduceByKey( lambda a, b: a+b) \
          .mapPartitions( lambda partition : sorted(partition, key=lambda kv: kv[1], reverse=True)[0:10])

# this is a creative way to sort each partition individually to find the top 10
# words within each partition, without a global sort; for this example, it's not
# relevant, but for big datasets, it's a much more efficient way as it takes
# advantage of the partitioning of RDDs and of parallel processing (i.e. it wor-
# ks within each partition, not requiring a global sort)

# Sort the top 10 words globally
  top10_words = top10_words_partitions.sortBy(lambda x: x[1], ascending = False) \
                .zipWithIndex() \
                .filter( lambda ranked: ranked[1] < 10) \
                .map( lambda ranked: ranked[0])
# Print top 10 words per partition
  print("Partitions: Top-10")
  for x in top10_words_partitions.glom().collect():
      print("{}".format(x))

# Print the final top 10 most frequent words
  print("Top-10 Most frequent words:")
  for x in top10_words.collect():
      print("{}".format(x))

# Stop the SparkContext
  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

##2. Weblog Analysis

Consider a set of log files captured during a DDOS (*Distributed Denial of Service*) attack, containing information for the web accesses performed during the attack to the server.

The log files contain text lines as shown below, with TAB as the separator:

date |IP_source | status_code | operation | URL | execution time |
-|-|-|-|-|-
timestamp  | string | int | string | string| float |
2016-12-06T08:58:35.318+0000|37.139.9.11|404|GET|/codemove/TTCENCUFMH3C|0.026

In [4]:
#@title Download the dataset
!wget -q -O web.log https://www.dropbox.com/s/0r8902uj9yum7dg/web.log?dl=0
!head -1 web.log

2016-12-06T08:58:35.318+0000 37.139.9.11 404 GET /codemove/TTCENCUFMH3C 0.026  


1) Count the number of unique IP addresses involved in the DDOS attack. Do not use the ***distinct()*** transformation.


In [6]:
import pyspark

sc = pyspark.SparkContext('local[*]')
try:
  # Read the web.log file and strip whitespace from each line
  lines = sc.textFile('web.log') \
          .map( lambda line: line.strip() )
  # Extract and filter unique IP addresses
  unique_ips = lines.map( lambda line: line.split()) \
          .filter( lambda values: len(values) == 6) \
          .map( lambda values : (values[1], None)) \
          .reduceByKey( lambda a, b : None ) \
          .map( lambda _ : (None, 1)) \
          .reduceByKey( lambda a, b : a+b ) \
  # Line by line:
    # 1- Split each line into a list of values
    # 2- Filter lines with exactly 6 values (excludes incomplete lines)
    # 3- Map each line to key-value pair (IP address - key, None - value)
    # 4- Reduce by key to keep only unique IP addresses (aggregates by keys - IPs, and discards the value - that's what 'lambda a, b: None' is doing)
    # 5- Map each unique IP address to (None, 1)
    # 6- Reduce by key (there is only one - None) to count the number of unique IP addresses

  # Display the count of unique IP addresses
  for _,c in unique_ips.collect(): # Loops through the unique_ips RDD
      print("{}".format(c))

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

167


With the .distinct transformation (which is easier to conceptualise)

In [7]:
import pyspark

sc = pyspark.SparkContext('local[*]')
try:
  lines = sc.textFile('web.log') \
          .map( lambda line: line.strip() )

  unique_ips = lines.map( lambda line: line.split()) \
          .filter( lambda values: len(values) == 6) \
          .map( lambda values : values[1] ) \
          .distinct() \
          .map( lambda _ : (None, 1)) \
          .reduceByKey( lambda a, b : a+b ) \


  for _,c in unique_ips.collect():
      print("{}".format(c))

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

167


In [8]:
import pyspark

sc = pyspark.SparkContext('local[*]')
try:
  lines = sc.textFile('web.log') \
          .map( lambda line: line.strip() )

  unique_ips = lines.map( lambda line: line.split()) \
          .filter( lambda values: len(values) == 6) \
          .map( lambda values : values[1] ) \
          .distinct()


  print(unique_ips.count())

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

167


2) For each interval of 10 seconds, provide the following information: [number of requests, average execution time, maximum time, minimum time]

In [10]:
import pyspark
from operator import *

sc = pyspark.SparkContext('local[*]')
try:
  lines = sc.textFile('web.log') \
         .map( lambda line: line.strip() )

  intervals = lines.map( lambda line: line.split()) \
          .filter( lambda values: len(values) == 6) \
          .map( lambda values: (values[0][0:18], float(values[5]))) \
          .map( lambda kv : (kv[0], (1, kv[1], kv[1], kv[1]))) \
          .reduceByKey( lambda a, b : (a[0] + b[0], a[1] + b[1], max(a[2],b[2]), min(a[3],b[3])) ) \
          .map( lambda kv : (kv[0], (kv[1][0], kv[1][1] / kv[1][0], kv[1][2], kv[1][3]))) \
          .sortByKey()

# 1- Splits the lines
# 2- Filters the lines, taking only the lines with 6 elements
# 3- Maps the lines - takes as key the characters 0 to 17 of the 0 index value
# (so the timestamp up to the tens of seconds digit), and passes as the value
# the execution time (as a float to allow calculations afterwards)
# 4- Maps each tuple to a standard structure where the key is the time interval,
# and the value is a tuple with a standard format: the count 1, sum equal to the
# value, max equal to the value, and min equal to the value.
# 5- Reduce by key - Reduces by key (time interval), aggregating the values. It
# sums counts, sums (for the mean), finds max and min for each time interval.
# 6-This .map operation is applied to each key-value pair (kv) in the RDD, where
# the key is the time interval, and the value is a tuple of counts, sum, max,
# and min associated with that time interval.
# The tuple returned by this map operation has the following structure:
# The key remains unchanged, representing the time interval.
# The value is a tuple containing:
    # Element 1 (kv[1][0]): Count of records for that time interval
    # (the count value in the original tuple).
    # Element 2 (kv[1][1] / kv[1][0]): Average value calculated by dividing the
    # sum (2nd element in the original tuple) by the count.
    # Element 3 (kv[1][2]): Maximum value (3rd element in the original tuple),
    # unchanged.
    # Element 4 (kv[1][3]): Minimum value (4th element in the original tuple),
    # unchanged.


  for interval in intervals.take(100):
    print(interval)

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

('2016-12-06T08:58:3', (483, 7.5934244306418215, 46.849, 0.013))
('2016-12-06T08:58:4', (2611, 30.159845653006503, 69.654, 0.014))
('2016-12-06T08:58:5', (5500, 38.52511163636371, 80.846, 0.017))
('2016-12-06T08:59:0', (6914, 38.53438212322824, 81.659, 0.018))
('2016-12-06T08:59:1', (6271, 32.96384978472328, 83.993, 0.017))
('2016-12-06T08:59:2', (5434, 17.29333143172616, 77.967, 0.051))
('2016-12-06T08:59:3', (8015, 11.210152214597631, 67.441, 0.056))
('2016-12-06T08:59:4', (7947, 7.761815779539431, 65.706, 0.914))
('2016-12-06T08:59:5', (5983, 3.821664382416849, 54.29, 0.678))
('2016-12-06T09:00:0', (6882, 8.649971519907023, 45.314, 0.017))
('2016-12-06T09:00:1', (9719, 7.857372672085602, 34.406, 0.225))
('2016-12-06T09:00:2', (6616, 4.611345223700128, 25.847, 0.014))
('2016-12-06T09:00:3', (6771, 1.6047638458130256, 26.53, 0.007))
('2016-12-06T09:00:4', (12135, 0.32576440049443844, 9.537, 0.007))
('2016-12-06T09:00:5', (8062, 0.20009092036715637, 1.905, 0.006))
('2016-12-06T09:01:0'

3) Create an inverted index that, for each interval of 10 seconds, has a list of (unique) IPs executing accesses (to each URL).

In [11]:
import pyspark

sc = pyspark.SparkContext('local[*]')
try:
  lines = sc.textFile('web.log') \
          .map( lambda line: line.strip() )


  intervals = lines.map( lambda line: line.split()) \
          .filter( lambda values: len(values) == 6) \
          .map( lambda values: ("{}-{}".format(values[0][0:18], values[4]), { values[1] } )) \
          .reduceByKey( lambda a, b : a | b ) \
          .sortByKey()

  for v in intervals.collect():
    print(v)

  sc.stop()
except Exception as e:
  print(e)
  sc.stop()

('2016-12-06T08:58:3-/codemove/01IX95N3AFP4', {'120.52.73.98'})
('2016-12-06T08:58:3-/codemove/0GLNQSHCISWJ', {'120.52.73.98'})
('2016-12-06T08:58:3-/codemove/1N80W0N2R36C', {'120.52.73.97'})
('2016-12-06T08:58:3-/codemove/1U6HCG3V2S9D', {'185.28.193.95'})
('2016-12-06T08:58:3-/codemove/2CEBGK8M78Y7', {'192.241.151.220'})
('2016-12-06T08:58:3-/codemove/5Q9SRR2G46PJ', {'120.52.73.98'})
('2016-12-06T08:58:3-/codemove/6GTXIA9YHX09', {'120.52.73.98'})
('2016-12-06T08:58:3-/codemove/7HIW17K7FDZI', {'97.77.104.22'})
('2016-12-06T08:58:3-/codemove/B35MFVKMU1C4', {'2002:894a:3a93:d:250:56ff:fe00:88c0'})
('2016-12-06T08:58:3-/codemove/BJJHJB8J8T7C', {'120.52.73.97'})
('2016-12-06T08:58:3-/codemove/BRPB8Y32OAGA', {'120.52.73.97'})
('2016-12-06T08:58:3-/codemove/CNDHDS3RE0QZ', {'202.47.236.252'})
('2016-12-06T08:58:3-/codemove/F1LSK225KYMF', {'211.140.26.58'})
('2016-12-06T08:58:3-/codemove/FJAQU6441E84', {'120.52.73.97'})
('2016-12-06T08:58:3-/codemove/GXZ23905JVVB', {'120.52.73.98'})
('2016-12-